In [1]:
import os
import time
import sys

import torch as t
from torch import nn
from torch.autograd import Variable

class simple_model(nn.Module):
    def __init__(self):
        super(simple_model, self).__init__()
        
        ####
        self.fc1 = nn.Linear(32, 24)
        self.fc1_1 = nn.Linear(24, 24)
        self.fc1_2 = nn.Linear(24, 24)
        self.fc2 = nn.Linear(24, 16)
        self.outLayer = nn.Linear(16, 3)
        self.softmax = nn.Softmax(-1)
        ####

    def forward(self, features):
        features = self.fc1(features)
        features = self.fc1_1(features)
        features = self.fc1_2(features)
        features = self.fc2(features)
        logits = self.outLayer(features)
        # print('output size', features.shape)
        probabs = self.softmax(logits)
        return logits, probabs
    

In [2]:
import pandas as pd
import numpy as np


In [11]:
mod2_df = pd.read_csv('mod2_df_v2.csv')

temp_csv = []
cnt = 0
for var in mod2_df.keys():
#     print(var, ':', mod2_df[var].isna().sum())
    if var not in {'PTID', 'DX', 'Last_DX', 'Years_bl', 'DX_bl', 'SITE', 'COLPROT', 'ORIGPROT'}:
        temp_csv += [var.split('_DUMMY_')[0]]
        cnt += 1
# print(mod2_df.keys())
temp_csv = set(temp_csv)
temp_csv, len(temp_csv)
temp_csv = ','.join(temp_csv)
temp_csv
with open('temp_csv.csv', 'w') as f:
    f.write(temp_csv)

In [100]:
SITEs = mod2_df['SITE'].unique()

THRES_SITE = int(SITEs.shape[0] * .2)
print('THRES_SITE:', THRES_SITE)

temp = (mod2_df['SITE'] == SITEs[0])
for site in SITEs[1:-THRES_SITE]:
    temp = temp | (mod2_df['SITE'] == site)
mod2_df_train = mod2_df[temp]

temp = (mod2_df['SITE'] == SITEs[-THRES_SITE])
for site in SITEs[-THRES_SITE+1:]:
    temp = temp | (mod2_df['SITE'] == site)
mod2_df_test = mod2_df[temp]

mod2_df_train.shape, mod2_df_test.shape

THRES_SITE: 13


((6782, 37), (761, 37))

In [101]:
PTID_list = mod2_df_train['PTID'].unique()
import random
random.shuffle(PTID_list)

THRES_PTID = int(PTID_list.shape[0] * .12)
print('THRES_PTID:', THRES_PTID)

temp = (mod2_df_train['PTID'] == PTID_list[0])
for PTID in PTID_list[1:-THRES_PTID]:
    temp = temp | (mod2_df_train['PTID'] == PTID)
mod2_df_train_1 = mod2_df_train[temp]

temp = (mod2_df_train['PTID'] == PTID_list[-THRES_PTID])
for PTID in PTID_list[-THRES_PTID+1:]:
    temp = temp | (mod2_df_train['PTID'] == PTID)
mod2_df_val = mod2_df_train[temp]

mod2_df_train = mod2_df_train_1

mod2_df_train.shape, mod2_df_val.shape

THRES_PTID: 192


((5952, 37), (830, 37))

In [102]:
DEBUG = False
means_std = {
    'mean' : np.zeros([mod2_df.shape[1]]),
    'std' : np.ones([mod2_df.shape[1]])
}
for i, var in enumerate(mod2_df.keys()):
    if var not in {'PTID', 'DX', 'Last_DX', 'DX_bl', 'COLPROT', 'ORIGPROT', 'FSVERSION', 'SITE', 'Years_bl'}:
        if DEBUG: 
            print('var:', var, mod2_df[var].count())
#         mod2_df[var] = mod2_df[var].fillna(mod2_df[var].mean())

        if mod2_df[var].unique().shape[0]>2:
            means_std['mean'][i] = mod2_df_train[var].mean()
            means_std['std'][i] = mod2_df_train[var].std()
        else: pass
#             print(mod2_df[var].unique())
#             means_std['mean'] += [0.]
#             means_std['std'] += [1.]
    else: pass
#             means_std['mean'] += [0.]
#             means_std['std'] += [1.]
            
means_std

{'mean': array([  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
         73.4703629 ,   0.        ,  16.10147849,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.52402554,
          1.51495296,   9.77813172,  15.37423387,   4.77436156,
         27.48655914,  36.77620968,   4.35584677,   4.30829973,
         56.16484695, 111.00352823,   3.87147177,  -4.70350463,
         -4.42900069]),
 'std': array([ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  6.95962662,  1.        ,
         2.77246426,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  0.64874067,  1.8528305 ,
         6.05593456,

In [211]:
label_map = {
            'CN': 0,
            'MCI': 1,
            'Dementia': 2
        }
print()
per_stage_count = {}
loss_weight_all = {}
for var1 in label_map:
    per_stage_count[var1] = {}
    loss_weight_all[var1] = np.zeros(3)
#     print('loss_weight_all[var1]:', loss_weight_all[var1].shape)
    for var2 in label_map:
        per_stage_count[var1][var2] = mod2_df_train[(mod2_df_train['DX']==var1) & (mod2_df_train['Last_DX']==var2)].shape[0]
        print(var1, '-->', var2, ': ', per_stage_count[var1][var2])
        loss_weight_all[var1][label_map[var2]] = 1. / per_stage_count[var1][var2]
    print()
    
per_stage_count, loss_weight_all


CN --> CN :  1695
CN --> MCI :  331
CN --> Dementia :  141

MCI --> CN :  203
MCI --> MCI :  1738
MCI --> Dementia :  910

Dementia --> CN :  1
Dementia --> MCI :  18
Dementia --> Dementia :  915



({'CN': {'CN': 1695, 'Dementia': 141, 'MCI': 331},
  'Dementia': {'CN': 1, 'Dementia': 915, 'MCI': 18},
  'MCI': {'CN': 203, 'Dementia': 910, 'MCI': 1738}},
 {'CN': array([0.00058997, 0.00302115, 0.0070922 ]),
  'Dementia': array([1.        , 0.05555556, 0.0010929 ]),
  'MCI': array([0.00492611, 0.00057537, 0.0010989 ])})

In [355]:
# patient_list = mod2_df_train['PTID'].unique()

# def get_sample(FEATURE_INDEX, df, STANDARDIZE, label_map, per_stage_count, means_std):
#     ## PTID patient_id for a patient who had multiple diagnosis 
#     FEATURE_STARTING = 8
# #     PTID = patient_list[_idx_]
# #     temp_df = mod2_df_train[mod2_df_train['PTID'] == PTID]
# #     assert temp_df.shape[0] > 1
# #     FEATURE_INDEX = np.random.choice(temp_df.shape[0])
#     features = df.iloc[FEATURE_INDEX, FEATURE_STARTING:]
#     features = features.to_numpy().astype(float)
#     if STANDARDIZE: 
#         features = (features - means_std['mean'][FEATURE_STARTING:]) / means_std['std'][FEATURE_STARTING:]
#     current_DX, last_DX = np.zeros([3]), np.zeros([3])
#     current_DX[label_map[df.iloc[FEATURE_INDEX, 1]]] = 1.        
#     last_DX[label_map[df.iloc[FEATURE_INDEX, 2]]] = 1.
#     loss_weight = 1. / per_stage_count[df.iloc[FEATURE_INDEX, 1]][df.iloc[FEATURE_INDEX, 2]] 
#     features = np.concatenate([current_DX, features])
#     return features, current_DX, last_DX, loss_weight

# features, current_DX, last_DX, loss_weight = get_sample(FEATURE_INDEX=1, 
#                                             df=mod2_df_train, 
#                                             STANDARDIZE=True, 
#                                             label_map=label_map, 
#                                             per_stage_count=per_stage_count,
#                                             means_std=means_std)
# features, current_DX, last_DX, features.shape, loss_weight

In [356]:
import torch
from torch.utils import data

class dataSet(data.Dataset):
    def __init__(self, df, loss_weight_all, means_std, dset_type='train'):
        super(dataSet, self).__init__()
        self.label_map = {
                          'CN': 0,
                          'MCI': 1,
                          'Dementia': 2
                         }
        self.df = df
        self.means_std = means_std
        self.loss_weight_all = loss_weight_all
        
    def __getitem__(self, index):
        features, _, label, loss_weight = self.get_sample(FEATURE_INDEX=index, 
                                                        df=self.df, 
                                                        STANDARDIZE=True, 
                                                        label_map=self.label_map, 
                                                        loss_weight_all=self.loss_weight_all,
                                                        means_std=self.means_std)
        features = torch.from_numpy(features).type(torch.FloatTensor)
        label = torch.from_numpy(label).type(torch.LongTensor)   
        loss_weight = torch.from_numpy(loss_weight).type(torch.FloatTensor)   
        return features, label, loss_weight
    
    def get_sample(self, FEATURE_INDEX, df, STANDARDIZE, label_map, loss_weight_all, means_std):
    ## PTID patient_id for a patient who had multiple diagnosis 
        FEATURE_STARTING = 8
        features = df.iloc[FEATURE_INDEX, FEATURE_STARTING:]
        features = features.to_numpy().astype(float)
        if STANDARDIZE: 
            features = (features - means_std['mean'][FEATURE_STARTING:]) / means_std['std'][FEATURE_STARTING:]
        current_DX, last_DX = np.zeros([3]), np.zeros([3])
        current_DX[label_map[df.iloc[FEATURE_INDEX, 1]]] = 1.        
        last_DX = np.array(label_map[df.iloc[FEATURE_INDEX, 2]]).astype(int)
        loss_weight = loss_weight_all[df.iloc[FEATURE_INDEX, 1]][label_map[df.iloc[FEATURE_INDEX, 2]]]
        features = np.concatenate([current_DX, features])
        return features, current_DX, last_DX, np.array([loss_weight])

    def __len__(self):
        return self.df.shape[0]

In [357]:
trainset = dataSet(df=mod2_df_train, means_std=means_std, loss_weight_all=loss_weight_all, dset_type='train')
testset = dataSet(df=mod2_df_test, means_std=means_std, loss_weight_all=loss_weight_all, dset_type='test')
valset = dataSet(df=mod2_df_val, means_std=means_std, loss_weight_all=loss_weight_all, dset_type='val')

In [358]:
trainset.__getitem__(739), loss_weight_all

((tensor([ 0.0000,  1.0000,  0.0000,  1.5273,  0.0000, -0.0366,  0.0000,
           1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  1.0000,
           0.0000,  1.0000,  0.0000,  0.0000, -0.8078, -0.0081,  0.0366,
           0.4021,  1.0842, -1.2720, -1.1525, -1.5685, -0.1199,  1.1570,
           0.7881, -0.3053, -0.9768, -1.1380]), tensor(2), tensor(1.00000e-03 *
         [ 1.0989])),
 {'CN': array([0.00058997, 0.00302115, 0.0070922 ]),
  'Dementia': array([1.        , 0.05555556, 0.0010929 ]),
  'MCI': array([0.00492611, 0.00057537, 0.0010989 ])})

In [375]:
# for s in [trainset, valset, testset]:
#     print(s.df.groupby('DX')['PTID'].nunique(), '\n->', s.df.groupby('Last_DX')['PTID'].nunique())
#     print()
batch_size = 16*8
num_workers = 0
train_loader = torch.utils.data.DataLoader(trainset,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          pin_memory=(torch.cuda.is_available()),
                                          num_workers=num_workers)
val_loader = torch.utils.data.DataLoader(valset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          pin_memory=(torch.cuda.is_available()),
                                          num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(testset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          pin_memory=(torch.cuda.is_available()),
                                          num_workers=num_workers)

In [377]:
loader = train_loader
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# DEVICE = 'cpu'

print(trainset.df.shape)

model = simple_model()
model = model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# criterion = nn.CrossEntropyLoss().to(DEVICE)

def epoch(model, optimizer, criterion, is_training, loader):
    pred = []
    true = []
    total_loss = 0.
    
    for batch_idx, (features, label, loss_weight) in enumerate(loader):
    #     print(features.shape, label.shape, loss_weight.shape)
        features = torch.autograd.Variable(features.to(DEVICE).float())
        label = torch.autograd.Variable(label.to(DEVICE).long())
        loss_weight = torch.autograd.Variable(loss_weight.to(DEVICE).float())

    #     print(features.shape, label.shape, loss_weight.shape)
        logits, probabs = model(features)
        log_logits = nn.functional.log_softmax(logits)
        
        if is_training:
            loss = nn.functional.cross_entropy(input=logits, target=label, reduce=False).to(DEVICE)
#             print(batch_idx, ':', loss, loss_weight)  
            loss = loss*loss_weight.reshape(-1)
            loss = loss.mean()
            total_loss += loss.item()
#             print(batch_idx, ':', loss)  

#             print(*zip(probabs.argmax(-1), label))

            ## compute gradient and do SGD step 
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        pred += probabs.argmax(-1).detach().cpu().numpy().tolist()
        true += label.detach().cpu().numpy().tolist()
        
    return np.array(pred), np.array(true), total_loss

pred, true, total_loss = epoch(model, optimizer, criterion, is_training=True, loader=train_loader)


(5952, 37)


C:\Users\Sazan\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
(pred == true).sum(), pred.shape[0]

(3565, 5952)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
EPOCH_NUM = 1000

for e in range(EPOCH_NUM):
    train_pred, train_true, train_loss = epoch(model, optimizer, criterion, 
                                   is_training=True, loader=train_loader)
    val_pred, val_true, val_loss = epoch(model, optimizer, criterion, 
                               is_training=False, loader=val_loader)
    
    p, r, f, s = precision_recall_fscore_support(y_true=val_true, y_pred=val_pred)
    train_acc = (train_pred==train_true).sum() / train_true.shape[0]
    val_acc = (val_pred==val_true).sum() / val_true.shape[0]
    print(e, ':', train_acc, val_acc)

C:\Users\Sazan\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


0 : 0.5524193548387096 0.3686746987951807
1 : 0.5549395161290323 0.5843373493975904
2 : 0.586861559139785 0.5831325301204819
3 : 0.5841733870967742 0.5156626506024097
4 : 0.5591397849462365 0.5240963855421686
5 : 0.5525873655913979 0.5807228915662651
6 : 0.6009744623655914 0.5903614457831325
7 : 0.5892137096774194 0.572289156626506
8 : 0.5919018817204301 0.5650602409638554
9 : 0.5877016129032258 0.5614457831325301
10 : 0.5895497311827957 0.5927710843373494
11 : 0.5971102150537635 0.608433734939759
12 : 0.6065188172043011 0.5879518072289157
13 : 0.5856854838709677 0.5108433734939759
14 : 0.5853494623655914 0.5795180722891566
15 : 0.5967741935483871 0.5301204819277109
16 : 0.5885416666666666 0.5831325301204819
17 : 0.5745967741935484 0.4783132530120482
18 : 0.5979502688172043 0.5566265060240964
19 : 0.5843413978494624 0.5108433734939759
20 : 0.5893817204301075 0.5843373493975904
21 : 0.6061827956989247 0.5409638554216868
22 : 0.5542674731182796 0.5891566265060241
24 : 0.5772849462365591 

194 : 0.5299059139784946 0.463855421686747
195 : 0.5470430107526881 0.5626506024096386
196 : 0.5411626344086021 0.5397590361445783
197 : 0.5623319892473119 0.5277108433734939
198 : 0.5702284946236559 0.6168674698795181
199 : 0.588877688172043 0.5048192771084338
200 : 0.525369623655914 0.572289156626506
201 : 0.5497311827956989 0.5349397590361445
202 : 0.5692204301075269 0.6289156626506024
203 : 0.5685483870967742 0.4939759036144578
204 : 0.5379704301075269 0.42771084337349397
205 : 0.5283938172043011 0.4939759036144578
206 : 0.5554435483870968 0.5746987951807229
207 : 0.5774529569892473 0.37349397590361444
208 : 0.5552755376344086 0.6036144578313253
209 : 0.5347782258064516 0.38072289156626504
210 : 0.5120967741935484 0.5445783132530121
211 : 0.5423387096774194 0.5831325301204819
212 : 0.572244623655914 0.572289156626506
213 : 0.573252688172043 0.5240963855421686
214 : 0.5596438172043011 0.4783132530120482
215 : 0.515625 0.5120481927710844
216 : 0.5646841397849462 0.5433734939759036
21

C:\Users\Sazan\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


349 : 0.5238575268817204 0.43373493975903615
350 : 0.5277217741935484 0.5180722891566265
351 : 0.5482190860215054 0.4734939759036145
352 : 0.4741263440860215 0.4963855421686747
353 : 0.5532594086021505 0.5349397590361445
354 : 0.5653561827956989 0.5216867469879518
355 : 0.5630040322580645 0.5204819277108433
356 : 0.46774193548387094 0.5048192771084338
357 : 0.5199932795698925 0.5493975903614458
358 : 0.5645161290322581 0.555421686746988
359 : 0.5655241935483871 0.5445783132530121
360 : 0.5552755376344086 0.6132530120481928
361 : 0.4986559139784946 0.4963855421686747
362 : 0.5697244623655914 0.5240963855421686
363 : 0.5658602150537635 0.572289156626506
364 : 0.5769489247311828 0.5783132530120482
365 : 0.5761088709677419 0.5578313253012048
366 : 0.571236559139785 0.4530120481927711
367 : 0.5425067204301075 0.35180722891566263
368 : 0.5285618279569892 0.5036144578313253
369 : 0.5505712365591398 0.5879518072289157
370 : 0.5816532258064516 0.4783132530120482
371 : 0.5848454301075269 0.62650

538 : 0.5974462365591398 0.5843373493975904
539 : 0.5816532258064516 0.5710843373493976
540 : 0.556619623655914 0.5783132530120482
541 : 0.6038306451612904 0.5771084337349398
542 : 0.5913978494623656 0.6072289156626506
543 : 0.5987903225806451 0.591566265060241
544 : 0.6113911290322581 0.5759036144578313
545 : 0.5631720430107527 0.5469879518072289
546 : 0.5863575268817204 0.5927710843373494
547 : 0.6050067204301075 0.5289156626506024
548 : 0.600638440860215 0.5578313253012048
549 : 0.5917338709677419 0.5662650602409639
550 : 0.603494623655914 0.555421686746988
551 : 0.6013104838709677 0.563855421686747
552 : 0.5954301075268817 0.5301204819277109
553 : 0.6058467741935484 0.5602409638554217
554 : 0.5971102150537635 0.5698795180722892
555 : 0.5991263440860215 0.5807228915662651
556 : 0.6090389784946236 0.5397590361445783
557 : 0.5814852150537635 0.563855421686747
558 : 0.6058467741935484 0.5602409638554217
559 : 0.613239247311828 0.5843373493975904
560 : 0.5991263440860215 0.6168674698795